### Imports

In [ ]:
import time
import json
import random
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

### Utils

In [ ]:
def get_random_string(k: int = 20):
    return "".join(random.choices("abcdefghijklmnopqrstuvwxyz", k=k))

def get_get_random_json():
    return {
        "str1": get_random_string(),
        "str2": get_random_string(),
        "int1": random.randint(0, 10_000),
        "int2": random.randint(-10_000, 0),
        "alphanumeric": f"{random.randint(0, 100_000)}" + get_random_string(),
    }

### Requests

In [ ]:
def hello_world_request(_):
    return requests.get("http://localhost:80/hello_world").content

def route_test_request(args):
    return requests.get(f"http://localhost:80/{args[0]}/{args[1]}").content

def json_parsing_request(payload):
    return requests.post("http://localhost:80/json", json=payload).content

In [ ]:
num_threads = [1, 2, 4, 8]
num_reqs = [1_000, 2_000, 4_000, 8_000]

In [ ]:
# results: tuple(task, threads, reqs, time)
results = []
for nt in num_threads:
    for nr in num_reqs:
        # hello world
        with ThreadPoolExecutor(nt) as ex:
            start = time.perf_counter()
            res = ex.map(hello_world_request, range(nr))
        stop = time.perf_counter()
        print(f"Served {nr:,} in: {(stop-start)*1_000}ms")
        results.append(("hello_world", nt, nr, (stop-start)*1_000))

        # route testing
        req_stuff = [(random.randint(0, 10_000), get_random_string()) for _ in range(nr)]
        with ThreadPoolExecutor(nt) as ex:
            start = time.perf_counter()
            res = ex.map(route_test_request, req_stuff)
        stop = time.perf_counter()
        print(f"Served {nr:,} in: {(stop-start)*1_000}ms")
        results.append(("route_testing", nt, nr, (stop-start)*1_000))

        # json parsing
        req_stuff = [(random.randint(0, 10_000), get_random_string()) for _ in range(nr)]
        with ThreadPoolExecutor(nt) as ex:
            start = time.perf_counter()
            res = ex.map(route_test_request, req_stuff)
        stop = time.perf_counter()
        print(f"Served {nr:,} in: {(stop-start)*1_000}ms")
        results.append(("json_parsing", nt, nr, (stop-start)*1_000))

In [ ]:
# write out 
df = pd.DataFrame(results)
df.columns = ["task", "threads", "requests", "time(ms)"]
df.to_csv("../results/fastapi_native.csv")